PROGRAM WRITTEN TO DOWNLOAD APPROPRIATE IN-SITU PLASMA DATA FROM SPACECRAFT IN EARTH'S MAGNETOSPHERE AND CALCULATE SHOCK SPEED.

written by: Axel Bernal 

In [1]:
#import libraries
import speasy as spz
from datetime import datetime
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math

In [2]:
#create speasy data trees
amda_tree = spz.inventories.tree.amda

cda_tree = spz.inventories.tree.cda

ssc_tree = spz.inventories.tree.ssc

In [3]:
#Data test used to remove unwanted spikes in plasma data. (NOT NEEDED IF DATA MUST BE RAW)
def data_test2(data):
    for i in range(0,len(data)):
        if abs(data[i]) > 10**15:                                             #spikes are usually spacecraft fill vals = 10^30
            count = 1
            k = i
            while k + 1 < len(data) and abs(data[k+1]) > 10**15:              #tests if spike is last data point or next data point is a spike
                count += 1
                k += 1
            else:
                #if last data point is spike Replace with last non-spike value
                if k == len(data) and i != 0:
                    for j in range(0,count):
                        data[i + j] = data[i-1]
                
                #if first value is a spike but not all values are spikes then Replace with first non-spike value
                elif i == 0 and count != len(data):
                    for j in range(0,count):
                        data[i + j] = data[k+1] 
                
                #if All vlaues are spikes then Replace with zeros     
                elif i == 0 and count == len(data):
                    data = np.zeros(len(data))
                    return data
                
                else:
                    #takes average between first and last non-spike value
                    data_end = data[k+1]
                    data_start = data[i-1]
                    data[i] = (data_start + data_end) / 2
                    i = k + 1

    return data

In [4]:
#function that will download proton density data from desired spacecraft for specified time interval (usually 8 minutes).

def get_density(spacecraft, date, interval):
    
    ## Setting Time Intervals for Shocks
    upstream_t1 = date - dt.timedelta(minutes=int(interval + 1))
    upstream_t2 = date - dt.timedelta(minutes=1)          #Set upstream time interval

    downstream_t3 = date + dt.timedelta(minutes=2)       #Set downstream time interval
    downstream_t4 = date + dt.timedelta(minutes=int(interval + 2))
    
    #print('Function: collecting density data from', spacecraft)
    
    #Collect data from desired spacecraft;
    if spacecraft == 'C1':
        cluster1_density_up = spz.get_data(cda_tree.Cluster.C1.CIS.C1_PP_CIS.N_p__C1_PP_CIS, upstream_t1, upstream_t2)
        density_data_up = cluster1_density_up.values
        
        cluster1_density_down = spz.get_data(cda_tree.Cluster.C1.CIS.C1_PP_CIS.N_p__C1_PP_CIS, downstream_t3, downstream_t4)
        density_data_down = cluster1_density_down.values
        
        return density_data_up, density_data_down
    
    if spacecraft == 'C3':
        cluster3_density_up = spz.get_data(cda_tree.Cluster.C3.CIS.C3_PP_CIS.N_p__C3_PP_CIS, upstream_t1, upstream_t2)
        density_data_up = cluster3_density_up.values
        
        cluster3_density_down = spz.get_data(cda_tree.Cluster.C3.CIS.C3_PP_CIS.N_p__C3_PP_CIS, downstream_t3, downstream_t4)
        density_data_down = cluster3_density_down.values
        
        return density_data_up, density_data_down
    
    if spacecraft == 'C4':
        cluster4_density_up = spz.get_data(cda_tree.Cluster.C4.CIS.C4_PP_CIS.N_p__C4_PP_CIS, upstream_t1, upstream_t2)
        density_data_up = cluster4_density_up.values
        
        cluster4_density_down = spz.get_data(cda_tree.Cluster.C4.CIS.C4_PP_CIS.N_p__C4_PP_CIS, downstream_t3, downstream_t4)
        density_data_down = cluster4_density_down.values
        
        return density_data_up, density_data_down
    
    
    if spacecraft == 'DS1':
        DS1_density_up = spz.get_data(amda_tree.Parameters.Double_Star.DoubleStar_1.HIA.dstar1_hia_prp.ds1_n, upstream_t1, upstream_t2)
        density_data_up = DS1_density_up.values
        
        DS1_density_down = spz.get_data(amda_tree.Parameters.Double_Star.DoubleStar_1.HIA.dstar1_hia_prp.ds1_n, downstream_t3, downstream_t4)
        density_data_down = DS1_density_down.values
        
        return density_data_up, density_data_down
    
    
    if spacecraft == 'THA':
        THA_density_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_A.ESA.tha_peem_all.tha_n_peem, upstream_t1, upstream_t2)
        density_data_up = THA_density_up.values
        
        THA_density_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_A.ESA.tha_peem_all.tha_n_peem, downstream_t3, downstream_t4)
        density_data_down = THA_density_down.values
        
        return density_data_up, density_data_down
    
    if spacecraft == 'THB':
        THB_density_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_B.ESA.thb_peem_all.thb_n_peem, upstream_t1, upstream_t2)
        density_data_up = THB_density_up.values
        
        THB_density_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_B.ESA.thb_peem_all.thb_n_peem, downstream_t3, downstream_t4)
        density_data_down = THB_density_down.values
        
        return density_data_up, density_data_down
    
    if spacecraft == 'THC':
        THC_density_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_C.ESA.thc_peem_all.thc_n_peem, upstream_t1, upstream_t2)
        density_data_up = THC_density_up.values
        
        THC_density_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_C.ESA.thc_peem_all.thc_n_peem, downstream_t3, downstream_t4)
        density_data_down = THC_density_down.values
        
        return density_data_up, density_data_down
    
    if spacecraft == 'THD':
        THD_density_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_D.ESA.thd_peem_all.thd_n_peem, upstream_t1, upstream_t2)
        density_data_up = THD_density_up.values
        
        THD_density_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_D.ESA.thd_peem_all.thd_n_peem, downstream_t3, downstream_t4)
        density_data_down = THD_density_down.values
        
        return density_data_up, density_data_down
    
    if spacecraft == 'THE':
        THE_density_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_E.ESA.the_peem_all.the_n_peem, upstream_t1, upstream_t2)
        density_data_up = THE_density_up.values
        
        THE_density_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_E.ESA.the_peem_all.the_n_peem, downstream_t3, downstream_t4)
        density_data_down = THE_density_down.values

        return density_data_up, density_data_down
    
    if spacecraft == 'MMS':
        MMS_density_up = spz.get_data(amda_tree.Parameters.MMS.MMS1.HPCA.mms1_hpca_moms.mms1_heplus_n, upstream_t1, upstream_t2)
        density_data_up = MMS_density_up.values
        
        MMS_density_down = spz.get_data(amda_tree.Parameters.MMS.MMS1.HPCA.mms1_hpca_moms.mms1_heplus_n, downstream_t3, downstream_t4)
        density_data_down = MMS_density_down.values
        
        return density_data_up, density_data_down
    
    if spacecraft == 'WIND':
        WIND_density_up = spz.get_data(cda_tree.Wind.WIND.SWE.WI_H1_SWE.Proton_Np_nonlin, upstream_t1, upstream_t2)
        density_data_up = WIND_density_up.values
        
        WIND_density_down = spz.get_data(cda_tree.Wind.WIND.SWE.WI_H1_SWE.Proton_Np_nonlin, downstream_t3, downstream_t4)
        density_data_down = WIND_density_down.values
        
        return density_data_up, density_data_down
    
    else: 
        
        print('Function failed: spacecraft not recognised')
        
        

In [5]:
#function to download B field data from desired spacecraft over specified time inetrval.

def get_Bfield(spacecraft, date, interval):
    
    ## Setting Time Intervals for Shocks
    upstream_t1 = date - dt.timedelta(minutes=int(interval + 1))
    upstream_t2 = date - dt.timedelta(minutes=1)          #Set upstream time interval

    downstream_t3 = date + dt.timedelta(minutes=2)       #Set downstream time interval
    downstream_t4 = date + dt.timedelta(minutes=int(interval + 2))

    
    #collect data from desired spacecraft;
    if spacecraft == 'C1':
        cluster1_B_up = spz.get_data(cda_tree.Cluster.C1.FGM_SPIN.C1_CP_FGM_SPIN.B_vec_xyz_gse__C1_CP_FGM_SPIN, upstream_t1, upstream_t2)
        
        #splits data into GSE coordinates
        Bx_up = cluster1_B_up.filter_columns(['Bx GSE']).values
        By_up = cluster1_B_up.filter_columns(['By GSE']).values
        Bz_up = cluster1_B_up.filter_columns(['Bz GSE']).values
        
        cluster1_B_down = spz.get_data(cda_tree.Cluster.C1.FGM_SPIN.C1_CP_FGM_SPIN.B_vec_xyz_gse__C1_CP_FGM_SPIN, downstream_t3, downstream_t4)
        
        Bx_down = cluster1_B_down.filter_columns(['Bx GSE']).values
        By_down = cluster1_B_down.filter_columns(['By GSE']).values
        Bz_down = cluster1_B_down.filter_columns(['Bz GSE']).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down
    
    if spacecraft == 'C3':
        cluster3_B_up = spz.get_data(cda_tree.Cluster.C3.FGM.C3_CP_FGM_SPIN.B_vec_xyz_gse__C3_CP_FGM_SPIN, upstream_t1, upstream_t2)
        
        Bx_up = cluster3_B_up.filter_columns(['Bx GSE']).values
        By_up = cluster3_B_up.filter_columns(['By GSE']).values
        Bz_up = cluster3_B_up.filter_columns(['Bz GSE']).values
        
        cluster3_B_down = spz.get_data(cda_tree.Cluster.C3.FGM.C3_CP_FGM_SPIN.B_vec_xyz_gse__C3_CP_FGM_SPIN, downstream_t3, downstream_t4)
        
        Bx_down = cluster3_B_down.filter_columns(['Bx GSE']).values
        By_down = cluster3_B_down.filter_columns(['By GSE']).values
        Bz_down = cluster3_B_down.filter_columns(['Bz GSE']).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down
    
    if spacecraft == 'C4':
        cluster4_B_up = spz.get_data(cda_tree.Cluster.C4.FGM.C4_CP_FGM_SPIN.B_vec_xyz_gse__C4_CP_FGM_SPIN, upstream_t1, upstream_t2)
        
        Bx_up = cluster4_B_up.filter_columns(['Bx GSE']).values
        By_up = cluster4_B_up.filter_columns(['By GSE']).values
        Bz_up = cluster4_B_up.filter_columns(['Bz GSE']).values
        
        cluster4_B_down = spz.get_data(cda_tree.Cluster.C4.FGM.C4_CP_FGM_SPIN.B_vec_xyz_gse__C4_CP_FGM_SPIN, downstream_t3, downstream_t4)
        
        Bx_down = cluster4_B_down.filter_columns(['Bx GSE']).values
        By_down = cluster4_B_down.filter_columns(['By GSE']).values
        Bz_down = cluster4_B_down.filter_columns(['Bz GSE']).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down
    
    
    if spacecraft == 'DS1':
        DS1_B_up = spz.get_data(amda_tree.Parameters.Double_Star.DoubleStar_1.FGM.dstar1_fgm_prp.ds1_b, upstream_t1, upstream_t2)
        
        Bx_up = DS1_B_up.filter_columns(['ds1_b[0]']).values
        By_up = DS1_B_up.filter_columns(['ds1_b[1]']).values
        Bz_up = DS1_B_up.filter_columns(['ds1_b[2]']).values
        
        DS1_B_down = spz.get_data(amda_tree.Parameters.Double_Star.DoubleStar_1.FGM.dstar1_fgm_prp.ds1_b, downstream_t3, downstream_t4)
        
        Bx_down = DS1_B_down.filter_columns(['ds1_b[0]']).values
        By_down = DS1_B_down.filter_columns(['ds1_b[1]']).values
        Bz_down = DS1_B_down.filter_columns(['ds1_b[2]']).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down
    
    
    if spacecraft == 'THA':
        THA_B_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_A.FGM.tha_fgm_l.tha_bl, upstream_t1, upstream_t2)
        
        Bx_up = THA_B_up.filter_columns(['tha_bl[0]']).values
        By_up = THA_B_up.filter_columns(['tha_bl[1]']).values
        Bz_up = THA_B_up.filter_columns(['tha_bl[2]']).values
        
        
        THA_B_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_A.FGM.tha_fgm_l.tha_bl, downstream_t3, downstream_t4)
        
        Bx_down = THA_B_down.filter_columns(['tha_bl[0]']).values
        By_down = THA_B_down.filter_columns(['tha_bl[1]']).values
        Bz_down = THA_B_down.filter_columns(['tha_bl[2]']).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down
    
    if spacecraft == 'THB':
        THB_B_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_B.FGM.thb_fgm_l.thb_bl, upstream_t1, upstream_t2)
        
        Bx_up = THB_B_up.filter_columns(['thb_bl[0]']).values
        By_up = THB_B_up.filter_columns(['thb_bl[1]']).values
        Bz_up = THB_B_up.filter_columns(['thb_bl[2]']).values
        
        
        THB_B_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_B.FGM.thb_fgm_l.thb_bl, downstream_t3, downstream_t4)
        
        Bx_down = THB_B_down.filter_columns(['thb_bl[0]']).values
        By_down = THB_B_down.filter_columns(['thb_bl[1]']).values
        Bz_down = THB_B_down.filter_columns(['thb_bl[2]']).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down
    
    if spacecraft == 'THC':
        THC_B_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_C.FGM.thc_fgm_l.thc_bl, upstream_t1, upstream_t2)
        
        Bx_up = THC_B_up.filter_columns(['thc_bl[0]']).values
        By_up = THC_B_up.filter_columns(['thc_bl[1]']).values
        Bz_up = THC_B_up.filter_columns(['thc_bl[2]']).values
        
        
        THC_B_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_C.FGM.thc_fgm_l.thc_bl, downstream_t3, downstream_t4)
        
        Bx_down = THC_B_down.filter_columns(['thc_bl[0]']).values
        By_down = THC_B_down.filter_columns(['thc_bl[1]']).values
        Bz_down = THC_B_down.filter_columns(['thc_bl[2]']).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down
    
    if spacecraft == 'THD':
        THD_B_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_D.FGM.thd_fgm_l.thd_bl, upstream_t1, upstream_t2)
        
        Bx_up = THD_B_up.filter_columns(['thd_bl[0]']).values
        By_up = THD_B_up.filter_columns(['thd_bl[1]']).values
        Bz_up = THD_B_up.filter_columns(['thd_bl[2]']).values
        
        
        THD_B_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_D.FGM.thd_fgm_l.thd_bl, downstream_t3, downstream_t4)
        
        Bx_down = THD_B_down.filter_columns(['thd_bl[0]']).values
        By_down = THD_B_down.filter_columns(['thd_bl[1]']).values
        Bz_down = THD_B_down.filter_columns(['thd_bl[2]']).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down
    
    if spacecraft == 'THE':
        THE_B_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_E.FGM.the_fgm_l.the_bl, upstream_t1, upstream_t2)
        
        Bx_up = THE_B_up.filter_columns(['the_bl[0]']).values
        By_up = THE_B_up.filter_columns(['the_bl[1]']).values
        Bz_up = THE_B_up.filter_columns(['the_bl[2]']).values
        
        
        THE_B_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_E.FGM.the_fgm_l.the_bl, downstream_t3, downstream_t4)
        
        Bx_down = THE_B_down.filter_columns(['the_bl[0]']).values
        By_down = THE_B_down.filter_columns(['the_bl[1]']).values
        Bz_down = THE_B_down.filter_columns(['the_bl[2]']).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down
    
    if spacecraft == 'MMS':
        MMS_B_up = spz.get_data(cda_tree.MMS.MMS1.FGM.MMS1_FGM_SRVY_L2.mms1_fgm_b_gse_srvy_l2, upstream_t1, upstream_t2)
        
        Bx_up = MMS_B_up.filter_columns(['Bx GSE']).values
        By_up = MMS_B_up.filter_columns(['By GSE']).values
        Bz_up = MMS_B_up.filter_columns(['Bz GSE']).values
        
        MMS_B_down = spz.get_data(cda_tree.MMS.MMS1.FGM.MMS1_FGM_SRVY_L2.mms1_fgm_b_gse_srvy_l2, downstream_t3, downstream_t4)
        
        Bx_down = MMS_B_down.filter_columns(['Bx GSE']).values
        By_down = MMS_B_down.filter_columns(['By GSE']).values
        Bz_down = MMS_B_down.filter_columns(['Bz GSE']).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down
    
    if spacecraft == 'WIND':
        WIND_B_upstream = spz.get_data(cda_tree.Wind.WIND.MFI.WI_H2_MFI.BGSE, upstream_t1, upstream_t2)
        
        Bx_up = WIND_B_upstream.filter_columns(["Bx (GSE)"]).values
        By_up = WIND_B_upstream.filter_columns(["By (GSE)"]).values
        Bz_up = WIND_B_upstream.filter_columns(["Bz (GSE)"]).values
        
        WIND_B_downstream = spz.get_data(cda_tree.Wind.WIND.MFI.WI_H2_MFI.BGSE, downstream_t3, downstream_t4)
        
        Bx_down = WIND_B_downstream.filter_columns(["Bx (GSE)"]).values
        By_down = WIND_B_downstream.filter_columns(["By (GSE)"]).values
        Bz_down = WIND_B_downstream.filter_columns(["Bz (GSE)"]).values
        
        return Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down

    
    else: 
        
        print('Function failed: spacecraft not recognised')
    

In [6]:
#function to donwload plasma bulk flow velocity from desired spacecraft over specified time interval.

def get_velocity(spacecraft, date, interval):

    ## Setting Time Intervals for Shocks
    upstream_t1 = date - dt.timedelta(minutes=int(interval + 1))
    upstream_t2 = date - dt.timedelta(minutes=1)          #Set upstream time interval

    downstream_t3 = date + dt.timedelta(minutes=2)       #Set downstream time interval
    downstream_t4 = date + dt.timedelta(minutes=int(interval + 2))
    
    # collect data from desired spacecraft;
    if spacecraft == 'C1':
        cluster1_v_up = spz.get_data(cda_tree.Cluster.C1.CIS.C1_PP_CIS.V_p_xyz_gse__C1_PP_CIS, upstream_t1, upstream_t2)

        #splits data into GSE coordinates
        Vx_up = cluster1_v_up.filter_columns(["Vx_p_gse"]).values
        Vy_up = cluster1_v_up.filter_columns(["Vy_p_gse"]).values
        Vz_up = cluster1_v_up.filter_columns(["Vz_p_gse"]).values
        
        cluster1_v_down = spz.get_data(cda_tree.Cluster.C1.CIS.C1_PP_CIS.V_p_xyz_gse__C1_PP_CIS, downstream_t3, downstream_t4)

        Vx_down = cluster1_v_down.filter_columns(["Vx_p_gse"]).values
        Vy_down = cluster1_v_down.filter_columns(["Vy_p_gse"]).values
        Vz_down = cluster1_v_down.filter_columns(["Vz_p_gse"]).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    if spacecraft == 'C3':
        cluster3_v_up = spz.get_data(cda_tree.Cluster.C3.CIS.C3_PP_CIS.V_p_xyz_gse__C3_PP_CIS, upstream_t1, upstream_t2)

        Vx_up = cluster3_v_up.filter_columns(["Vx_p_gse"]).values
        Vy_up = cluster3_v_up.filter_columns(["Vy_p_gse"]).values
        Vz_up = cluster3_v_up.filter_columns(["Vz_p_gse"]).values
        
        cluster3_v_down = spz.get_data(cda_tree.Cluster.C3.CIS.C3_PP_CIS.V_p_xyz_gse__C3_PP_CIS, downstream_t3, downstream_t4)

        Vx_down = cluster3_v_down.filter_columns(["Vx_p_gse"]).values
        Vy_down = cluster3_v_down.filter_columns(["Vy_p_gse"]).values
        Vz_down = cluster3_v_down.filter_columns(["Vz_p_gse"]).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    if spacecraft == 'C4':
        cluster4_v_up = spz.get_data(cda_tree.Cluster.C4.CIS.C4_PP_CIS.V_p_xyz_gse__C4_PP_CIS, upstream_t1, upstream_t2)

        Vx_up = cluster4_v_up.filter_columns(["Vx_p_gse"]).values
        Vy_up = cluster4_v_up.filter_columns(["Vy_p_gse"]).values
        Vz_up = cluster4_v_up.filter_columns(["Vz_p_gse"]).values
        
        cluster4_v_down = spz.get_data(cda_tree.Cluster.C4.CIS.C4_PP_CIS.V_p_xyz_gse__C4_PP_CIS, downstream_t3, downstream_t4)

        Vx_down = cluster4_v_down.filter_columns(["Vx_p_gse"]).values
        Vy_down = cluster4_v_down.filter_columns(["Vy_p_gse"]).values
        Vz_down = cluster4_v_down.filter_columns(["Vz_p_gse"]).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    
    if spacecraft == 'DS1':
        DS1_v_up = spz.get_data(amda_tree.Parameters.Double_Star.DoubleStar_1.HIA.dstar1_hia_prp.ds1_v, upstream_t1, upstream_t2)

        Vx_up = DS1_v_up.filter_columns(["ds1_v[0]"]).values
        Vy_up = DS1_v_up.filter_columns(["ds1_v[1]"]).values
        Vz_up = DS1_v_up.filter_columns(["ds1_v[2]"]).values
        
        DS1_v_down = spz.get_data(amda_tree.Parameters.Double_Star.DoubleStar_1.HIA.dstar1_hia_prp.ds1_v, downstream_t3, downstream_t4)

        Vx_down = DS1_v_down.filter_columns(["ds1_v[0]"]).values
        Vy_down = DS1_v_down.filter_columns(["ds1_v[1]"]).values
        Vz_down = DS1_v_down.filter_columns(["ds1_v[2]"]).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    
    if spacecraft == 'THA':
        THA_v_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_A.ESA.tha_peim_all.tha_v_peim, upstream_t1, upstream_t2)
        
        Vx_up = THA_v_up.filter_columns(['tha_v_peim[0]']).values
        Vy_up = THA_v_up.filter_columns(['tha_v_peim[1]']).values
        Vz_up = THA_v_up.filter_columns(['tha_v_peim[2]']).values
        
        
        THA_v_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_A.ESA.tha_peim_all.tha_v_peim, downstream_t3, downstream_t4)
        
        Vx_down = THA_v_down.filter_columns(['tha_v_peim[0]']).values
        Vy_down = THA_v_down.filter_columns(['tha_v_peim[1]']).values
        Vz_down = THA_v_down.filter_columns(['tha_v_peim[2]']).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    if spacecraft == 'THB':
        THB_v_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_B.ESA.thb_peim_all.thb_v_peim, upstream_t1, upstream_t2)
        
        Vx_up = THB_v_up.filter_columns(['thb_v_peim[0]']).values
        Vy_up = THB_v_up.filter_columns(['thb_v_peim[1]']).values
        Vz_up = THB_v_up.filter_columns(['thb_v_peim[2]']).values
        
        
        THB_v_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_B.ESA.thb_peim_all.thb_v_peim, downstream_t3, downstream_t4)
        
        Vx_down = THB_v_down.filter_columns(['thb_v_peim[0]']).values
        Vy_down = THB_v_down.filter_columns(['thb_v_peim[1]']).values
        Vz_down = THB_v_down.filter_columns(['thb_v_peim[2]']).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    if spacecraft == 'THC':
        THC_v_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_C.ESA.thc_peim_all.thc_v_peim, upstream_t1, upstream_t2)
        
        Vx_up = THC_v_up.filter_columns(['thc_v_peim[0]']).values
        Vy_up = THC_v_up.filter_columns(['thc_v_peim[1]']).values
        Vz_up = THC_v_up.filter_columns(['thc_v_peim[2]']).values
        
        
        THC_v_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_C.ESA.thc_peim_all.thc_v_peim, downstream_t3, downstream_t4)
        
        Vx_down = THC_v_down.filter_columns(['thc_v_peim[0]']).values
        Vy_down = THC_v_down.filter_columns(['thc_v_peim[1]']).values
        Vz_down = THC_v_down.filter_columns(['thc_v_peim[2]']).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    if spacecraft == 'THD':
        THD_v_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_D.ESA.thd_peim_all.thd_v_peim, upstream_t1, upstream_t2)
        
        Vx_up = THD_v_up.filter_columns(['thd_v_peim[0]']).values
        Vy_up = THD_v_up.filter_columns(['thd_v_peim[1]']).values
        Vz_up = THD_v_up.filter_columns(['thd_v_peim[2]']).values
        
        
        THD_v_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_D.ESA.thd_peim_all.thd_v_peim, downstream_t3, downstream_t4)
        
        Vx_down = THD_v_down.filter_columns(['thd_v_peim[0]']).values
        Vy_down = THD_v_down.filter_columns(['thd_v_peim[1]']).values
        Vz_down = THD_v_down.filter_columns(['thd_v_peim[2]']).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    if spacecraft == 'THE':
        THE_v_up = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_E.ESA.the_peim_all.the_v_peim, upstream_t1, upstream_t2)
        
        Vx_up = THE_v_up.filter_columns(['the_v_peim[0]']).values
        Vy_up = THE_v_up.filter_columns(['the_v_peim[1]']).values
        Vz_up = THE_v_up.filter_columns(['the_v_peim[2]']).values
        
        
        THE_v_down = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_E.ESA.the_peim_all.the_v_peim, downstream_t3, downstream_t4)
        
        Vx_down = THE_v_down.filter_columns(['the_v_peim[0]']).values
        Vy_down = THE_v_down.filter_columns(['the_v_peim[1]']).values
        Vz_down = THE_v_down.filter_columns(['the_v_peim[2]']).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    if spacecraft == 'MMS':
        MMS_v_up = spz.get_data(amda_tree.Parameters.MMS.MMS1.HPCA.mms1_hpca_moms.mms1_heplus_vgse, upstream_t1, upstream_t2)

        Vx_up = MMS_v_up.filter_columns(["mms1_heplus_vgse[0]"]).values
        Vy_up = MMS_v_up.filter_columns(["mms1_heplus_vgse[1]"]).values
        Vz_up = MMS_v_up.filter_columns(["mms1_heplus_vgse[2]"]).values
        
        MMS_v_down = spz.get_data(amda_tree.Parameters.MMS.MMS1.HPCA.mms1_hpca_moms.mms1_heplus_vgse, downstream_t3, downstream_t4)

        Vx_down = MMS_v_down.filter_columns(["mms1_heplus_vgse[0]"]).values
        Vy_down = MMS_v_down.filter_columns(["mms1_heplus_vgse[1]"]).values
        Vz_down = MMS_v_down.filter_columns(["mms1_heplus_vgse[2]"]).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    if spacecraft == 'WIND':
        Vx_up = spz.get_data(cda_tree.Wind.WIND.SWE.WI_H1_SWE.Proton_VX_nonlin, upstream_t1, upstream_t2).values
        Vy_up = spz.get_data(cda_tree.Wind.WIND.SWE.WI_H1_SWE.Proton_VY_nonlin, upstream_t1, upstream_t2).values
        Vz_up = spz.get_data(cda_tree.Wind.WIND.SWE.WI_H1_SWE.Proton_VZ_nonlin, upstream_t1, upstream_t2).values
        
        Vx_down = spz.get_data(cda_tree.Wind.WIND.SWE.WI_H1_SWE.Proton_VX_nonlin, downstream_t3, downstream_t4).values
        Vy_down = spz.get_data(cda_tree.Wind.WIND.SWE.WI_H1_SWE.Proton_VY_nonlin, downstream_t3, downstream_t4).values
        Vz_down = spz.get_data(cda_tree.Wind.WIND.SWE.WI_H1_SWE.Proton_VZ_nonlin, downstream_t3, downstream_t4).values
        
        return Vx_up, Vy_up, Vz_up, Vx_down, Vy_down, Vz_down
    
    else: 
        
        print('Function failed: spacecraft not recognised')
    

In [7]:
#Function to calculate shock normal and speed.

def shock_velocity(spacecraft, date, interval):
    print('Function: starting data collection')
    ## Setting Time Intervals for Shocks ##
    
    ## Setting Time Intervals for Shocks
    upstream_t1 = date - dt.timedelta(minutes=int(interval + 1))
    upstream_t2 = date - dt.timedelta(minutes=1)          #Set upstream time interval

    downstream_t3 = date + dt.timedelta(minutes=2)       #Set downstream time interval
    downstream_t4 = date + dt.timedelta(minutes=int(interval + 2))
        

    ## Get Mean Proton Density Data ##
    density_upstreamRAW, density_downstreamRAW = get_density(spacecraft, date, interval)
    
    #Data Test for upstream density
    if len(density_upstreamRAW)==0:
        print('Function FAIL: No upstream density data')
        v_shock = 0
        shock_n = 0
        return v_shock, shock_n
    
    #Data test for downstream density
    if len(density_downstreamRAW)==0:
        print('Function FAIL: No downstream density data')
        v_shock = 0
        shock_n = 0
        return v_shock, shock_n
    
    #2nd data test to remove artifical spikes
    density_upstream = data_test2(density_upstreamRAW)
    del density_upstreamRAW                                                    #removing old array for efficiency
    
    density_downstream = data_test2(density_downstreamRAW)
    del density_downstreamRAW

    #TAKE MEAN VALUES AND STORE
    n_mean_upstream = np.nanmean(density_upstream)
    del density_upstream
    
    n_mean_downstream = np.nanmean(density_downstream)
    del density_downstream


    ## Gather B field data as upstream and downstream components ##
    Bx_up, By_up, Bz_up, Bx_down, By_down, Bz_down = get_Bfield(spacecraft, date, interval)
    
    #Data Test
    if len(Bx_up)==0:
        print('Function FAIL: No upstream B data')
        v_shock = 0
        shock_n = 0
        return v_shock, shock_n
    
    if len(Bx_down)==0:
        print('Function FAIL: No downstream B data')
        v_shock = 0
        shock_n = 0
        return v_shock, shock_n

    #Split into components
    ## UPSTREAM ##
    Bx_up_mean = np.nanmean(Bx_up)
    By_up_mean = np.nanmean(By_up)
    Bz_up_mean = np.nanmean(Bz_up)
    
    #delete old arrays
    del Bx_up
    del By_up
    del Bz_up

    B_up_data = np.array([Bx_up_mean,By_up_mean,Bz_up_mean])

    
    ## DOWNSTREAM ##
    Bx_down_mean = np.nanmean(Bx_down)
    By_down_mean = np.nanmean(By_down)
    Bz_down_mean = np.nanmean(Bz_down)
    
    del Bx_down
    del By_down
    del Bz_down

    B_down_data = np.array([Bx_down_mean,By_down_mean,Bz_down_mean])

    # Get Velocity Data
    ## UPSTREAM ##
    Vx_upRAW, Vy_upRAW, Vz_upRAW, Vx_downRAW, Vy_downRAW, Vz_downRAW = get_velocity(spacecraft, date, interval)
    
    if len(Vx_upRAW)==0:
        print('Function FAIL: No upstream V data')
        mach_number = 0
        return v_shock, mach_number
    
    Vx_up = data_test2(Vx_upRAW)
    del Vx_upRAW
    Vy_up = data_test2(Vy_upRAW)
    del Vy_upRAW
    Vz_up = data_test2(Vz_upRAW)
    del Vz_upRAW
   
    Vx_up_mean = np.nanmean(Vx_up)
    Vy_up_mean = np.nanmean(Vy_up)
    Vz_up_mean = np.nanmean(Vz_up)
    
    del Vx_up
    del Vy_up
    del Vz_up

    V_up_data = np.array([Vx_up_mean,Vy_up_mean,Vz_up_mean])

    ## Downstream ##
    if len(Vx_downRAW)==0:
        print('Function FAIL: No downstream V data')
        mach_number = 0
        return v_shock, mach_number

    Vx_down = data_test2(Vx_downRAW)
    del Vx_downRAW
    Vy_down = data_test2(Vy_downRAW)
    del Vy_downRAW
    Vz_down = data_test2(Vz_downRAW)
    del Vz_downRAW
    
    Vx_down_mean = np.nanmean(Vx_down)
    Vy_down_mean = np.nanmean(Vy_down)
    Vz_down_mean = np.nanmean(Vz_down)
    
    del Vx_down
    del Vy_down
    del Vz_down

    V_down_data = np.array([Vx_down_mean,Vy_down_mean,Vz_down_mean])
        
    print('Function: data collection complete')
    print(' ')
    # GET SHOCK NORMAL
    shock_n = np.cross(B_down_data - B_up_data, np.cross(B_down_data - B_up_data, V_down_data - V_up_data))

    shock_n = -shock_n/np.linalg.norm(shock_n)         #This ensures the shock normal has correct sign and is normalised

    # Now calculate velocity of shock 
    v_shock = abs(np.dot(((n_mean_downstream * V_down_data)-(n_mean_upstream * V_up_data))/(n_mean_downstream - n_mean_upstream), shock_n)) 
    
    return v_shock, shock_n
    

In [8]:
# Create a plotting function

def plot_data(spacecraft,date):
    
    # Set data collection time frame taking into account large time frame for manual shock searching
    start_time = date - dt.timedelta(minutes=30)
    end_time = date + dt.timedelta(minutes=30)
    
    ## Setting Time Intervals for Shocks
    upstream_t1 = date - dt.timedelta(minutes=9)
    upstream_t2 = date - dt.timedelta(minutes=1)          #Set upstream time interval (8 minutes)

    downstream_t3 = date + dt.timedelta(minutes=2)       #Set downstream time interval (8 minutes)
    downstream_t4 = date + dt.timedelta(minutes=10)
    
    #Check spacecraft;
    print('Function: collecting data for plotting')
    if spacecraft == 'C1':
        #B field magnitude
        B_field = spz.get_data(cda_tree.Cluster.C1.FGM_SPIN.C1_CP_FGM_SPIN.B_mag__C1_CP_FGM_SPIN, start_time, end_time)
        
        #B field componenets
        B_field_comps = spz.get_data(cda_tree.Cluster.C1.FGM_SPIN.C1_CP_FGM_SPIN.B_vec_xyz_gse__C1_CP_FGM_SPIN, start_time, end_time)
        
        #splits data into GSE coordinates
        Bx = B_field_comps.filter_columns(['Bx GSE']).values
        By = B_field_comps.filter_columns(['By GSE']).values
        Bz = B_field_comps.filter_columns(['Bz GSE']).values
        
        #Proton density
        density = spz.get_data(cda_tree.Cluster.C1.CIS.C1_PP_CIS.N_p__C1_PP_CIS, start_time, end_time)
        
        #Bulk flow velocity
        velocity = spz.get_data(cda_tree.Cluster.C1.CIS.C1_PP_CIS.V_p_xyz_gse__C1_PP_CIS, start_time, end_time)

        #splits into GSE coordinates
        Vx = velocity.filter_columns(["Vx_p_gse"]).values
        Vy = velocity.filter_columns(["Vy_p_gse"]).values
        Vz = velocity.filter_columns(["Vz_p_gse"]).values

        #gets magnitude
        velocity_mag = np.sqrt(Vx**2 + Vy**2 + Vz**2)    
        
    
    if spacecraft == 'C3':
        B_field = spz.get_data(cda_tree.Cluster.C3.FGM.C3_CP_FGM_SPIN.B_mag__C3_CP_FGM_SPIN, start_time, end_time)
        
        B_field_comps = spz.get_data(cda_tree.Cluster.C3.FGM_SPIN.C3_CP_FGM_SPIN.B_vec_xyz_gse__C3_CP_FGM_SPIN, start_time, end_time)
        
        #splits data into GSE coordinates
        Bx = B_field_comps.filter_columns(['Bx GSE']).values
        By = B_field_comps.filter_columns(['By GSE']).values
        Bz = B_field_comps.filter_columns(['Bz GSE']).values
        
        density = spz.get_data(cda_tree.Cluster.C3.CIS.C3_PP_CIS.N_p__C3_PP_CIS, start_time, end_time)
        
        velocity = spz.get_data(cda_tree.Cluster.C3.CIS.C3_PP_CIS.V_p_xyz_gse__C3_PP_CIS, start_time, end_time)

        Vx = velocity.filter_columns(["Vx_p_gse"]).values
        Vy = velocity.filter_columns(["Vy_p_gse"]).values
        Vz = velocity.filter_columns(["Vz_p_gse"]).values

        velocity_mag = np.sqrt(Vx**2 + Vy**2 + Vz**2)
        
    if spacecraft == 'C4':
        B_field = spz.get_data(cda_tree.Cluster.C4.FGM.C4_CP_FGM_SPIN.B_mag__C4_CP_FGM_SPIN, start_time, end_time)
        
        B_field_comps = spz.get_data(cda_tree.Cluster.C4.FGM_SPIN.C4_CP_FGM_SPIN.B_vec_xyz_gse__C4_CP_FGM_SPIN, start_time, end_time)
        
        #splits data into GSE coordinates
        Bx = B_field_comps.filter_columns(['Bx GSE']).values
        By = B_field_comps.filter_columns(['By GSE']).values
        Bz = B_field_comps.filter_columns(['Bz GSE']).values
        
        density = spz.get_data(cda_tree.Cluster.C4.CIS.C4_PP_CIS.N_p__C4_PP_CIS, start_time, end_time)
        
        velocity = spz.get_data(cda_tree.Cluster.C4.CIS.C4_PP_CIS.V_p_xyz_gse__C4_PP_CIS, start_time, end_time)

        Vx = velocity.filter_columns(["Vx_p_gse"]).values
        Vy = velocity.filter_columns(["Vy_p_gse"]).values
        Vz = velocity.filter_columns(["Vz_p_gse"]).values

        velocity_mag = np.sqrt(Vx**2 + Vy**2 + Vz**2)
        
        
    if spacecraft == 'DS1':
        B_field = spz.get_data(amda_tree.Parameters.Double_Star.DoubleStar_1.FGM.dstar1_fgm_prp.dstar1_fgm_prp_bt, start_time, end_time)
        
        B_field_comps = spz.get_data(amda_tree.Parameters.Double_Star.DoubleStar_1.FGM.dstar1_fgm_prp.ds1_b, start_time, end_time)
        
        Bx = B_field_comps.filter_columns(['ds1_b[0]']).values
        By = B_field_comps.filter_columns(['ds1_b[1]']).values
        Bz = B_field_comps.filter_columns(['ds1_b[2]']).values
        
        density = spz.get_data(amda_tree.Parameters.Double_Star.DoubleStar_1.HIA.dstar1_hia_prp.ds1_n, start_time, end_time)
        
        velocity = spz.get_data(amda_tree.Parameters.Double_Star.DoubleStar_1.HIA.dstar1_hia_prp.ds1_v, start_time, end_time)
        
        Vx = velocity.filter_columns(["ds1_v[0]"]).values
        Vy = velocity.filter_columns(["ds1_v[1]"]).values
        Vz = velocity.filter_columns(["ds1_v[2]"]).values

        velocity_mag = np.sqrt(Vx**2 + Vy**2 + Vz**2)
        
        
    if spacecraft == 'THA':
        B_field = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_A.FGM.tha_fgm_l.tha_bl_tot, start_time, end_time)
        
        B_field_comps = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_A.FGM.tha_fgm_l.tha_bl, start_time, end_time)
        
        Bx = B_field_comps.filter_columns(['tha_bl[0]']).values
        By = B_field_comps.filter_columns(['tha_bl[1]']).values
        Bz = B_field_comps.filter_columns(['tha_bl[2]']).values
        
        density = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_A.ESA.tha_peem_all.tha_n_peem, start_time, end_time)
        
        velocity = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_A.ESA.tha_peim_all.tha_v_peim_tot, start_time, end_time)
        velocity_mag = velocity.values
        
        
    if spacecraft == 'THB':
        B_field = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_B.FGM.thb_fgm_l.thb_bl_tot, start_time, end_time)
        
        B_field_comps = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_B.FGM.thb_fgm_l.thb_bl, start_time, end_time)
        
        Bx = B_field_comps.filter_columns(['tha_bl[0]']).values
        By = B_field_comps.filter_columns(['tha_bl[1]']).values
        Bz = B_field_comps.filter_columns(['tha_bl[2]']).values
        
        density = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_B.ESA.thb_peem_all.thb_n_peem, start_time, end_time)
        
        velocity = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_B.ESA.thb_peim_all.thb_v_peim_tot, start_time, end_time)
        velocity_mag = velocity.values
        
    if spacecraft == 'THC':
        B_field = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_C.FGM.thc_fgm_l.thc_bl_tot, start_time, end_time)
        
        B_field_comps = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_C.FGM.thc_fgm_l.thc_bl, start_time, end_time)
        
        Bx = B_field_comps.filter_columns(['tha_bl[0]']).values
        By = B_field_comps.filter_columns(['tha_bl[1]']).values
        Bz = B_field_comps.filter_columns(['tha_bl[2]']).values
        
        density = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_C.ESA.thc_peem_all.thc_n_peem, start_time, end_time)
        
        velocity = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_C.ESA.thc_peim_all.thc_v_peim_tot, start_time, end_time)
        velocity_mag = velocity.values
        
    if spacecraft == 'THD':
        B_field = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_D.FGM.thd_fgm_l.thd_bl_tot, start_time, end_time)
        
        B_field_comps = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_D.FGM.thd_fgm_l.thd_bl, start_time, end_time)
        
        Bx = B_field_comps.filter_columns(['thd_bl[0]']).values
        By = B_field_comps.filter_columns(['thd_bl[1]']).values
        Bz = B_field_comps.filter_columns(['thd_bl[2]']).values
        
        density = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_D.ESA.thd_peem_all.thd_n_peem, start_time, end_time)
        
        velocity = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_D.ESA.thd_peim_all.thd_v_peim_tot, start_time, end_time)
        velocity_mag = velocity.values
    
    if spacecraft == 'THE':
        B_field = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_E.FGM.the_fgm_l.the_bl_tot, start_time, end_time)
        
        B_field_comps = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_E.FGM.the_fgm_l.the_bl, start_time, end_time)
        
        Bx = B_field_comps.filter_columns(['the_bl[0]']).values
        By = B_field_comps.filter_columns(['the_bl[1]']).values
        Bz = B_field_comps.filter_columns(['the_bl[2]']).values
        
        density = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_E.ESA.the_peem_all.the_n_peem, start_time, end_time)
        
        velocity = spz.get_data(amda_tree.Parameters.THEMIS.THEMIS_E.ESA.the_peim_all.the_v_peim_tot, start_time, end_time) 
        velocity_mag = velocity.values
    
    if spacecraft == 'MMS':
        
        MMS_B = spz.get_data(cda_tree.MMS.MMS1.FGM.MMS1_FGM_SRVY_L2.mms1_fgm_b_gse_srvy_l2, start_time, end_time)
        B_field = MMS_B.filter_columns(['Bt'])
        
        B_field_comps = spz.get_data(cda_tree.MMS.MMS1.FGM.MMS1_FGM_SRVY_L2.mms1_fgm_b_gse_srvy_l2, start_time, end_time)
        
        Bx = B_field_comps.filter_columns(['Bx GSE']).values
        By = B_field_comps.filter_columns(['By GSE']).values
        Bz = B_field_comps.filter_columns(['Bz GSE']).values
        
        density = spz.get_data(amda_tree.Parameters.MMS.MMS1.HPCA.mms1_hpca_moms.mms1_heplus_n, start_time, end_time)
        
        velocity = spz.get_data(amda_tree.Parameters.MMS.MMS1.HPCA.mms1_hpca_moms.mms1_heplus_vgse, start_time, end_time)

        Vx = velocity.filter_columns(["mms1_heplus_vgse[0]"]).values
        Vy = velocity.filter_columns(["mms1_heplus_vgse[1]"]).values
        Vz = velocity.filter_columns(["mms1_heplus_vgse[2]"]).values

        velocity_mag = np.sqrt(Vx**2 + Vy**2 + Vz**2)
        
    if spacecraft == 'WIND':
        
        B_field = spz.get_data(cda_tree.Wind.WIND.MFI.WI_H2_MFI.BF1, start_time, end_time)
        
        B_field_comps = spz.get_data(cda_tree.Wind.WIND.MFI.WI_H2_MFI.BGSE, start_time, end_time)
        
        Bx = B_field_comps.filter_columns(["Bx (GSE)"]).values
        By = B_field_comps.filter_columns(["By (GSE)"]).values
        Bz = B_field_comps.filter_columns(["Bz (GSE)"]).values
        
        density = spz.get_data(cda_tree.Wind.WIND.SWE.WI_H1_SWE.Proton_Np_nonlin ,start_time, end_time)
        
        velocity = spz.get_data(cda_tree.Wind.WIND.SWE.WI_H1_SWE.Proton_V_moment ,start_time, end_time)
        velocity_mag = velocity.values
        
    print('Function: data collection complete')
    
    #Now create plots
    fig = plt.figure(figsize=(11, 9))
    gs = fig.add_gridspec(4, hspace=0)
    axes = gs.subplots(sharex=True, sharey=False)
    
    plt.xlabel('Universal Time (date/hour/minute)',fontsize = 16)

    axes[0].plot(B_field.time, B_field.values, color='black')
    axes[0].axvline(date, c = 'red')
    axes[0].axvline(upstream_t1, color='black', linestyle='--')
    axes[0].axvline(upstream_t2, color='black', linestyle='--')
    axes[0].axvline(downstream_t3, color='black', linestyle='--')
    axes[0].axvline(downstream_t4, color='black', linestyle='--')
    axes[0].set_ylabel('B Field (nT)', fontsize = 16)
    
    axes[1].plot(B_field_comps.time, Bx, color='blue', label = 'Bx')
    axes[1].plot(B_field_comps.time, By, color='orange', label = 'By')
    axes[1].plot(B_field_comps.time, Bz, color='yellow', label = 'Bz')
    axes[1].legend()
    axes[1].axvline(date, c = 'red')
    axes[1].axvline(upstream_t1, color='black', linestyle='--')
    axes[1].axvline(upstream_t2, color='black', linestyle='--')
    axes[1].axvline(downstream_t3, color='black', linestyle='--')
    axes[1].axvline(downstream_t4, color='black', linestyle='--')
    axes[1].set_ylabel('B Field GSE (nT)', fontsize = 16)

    axes[2].plot(velocity.time, velocity_mag, color='black')
    axes[2].axvline(date, c = 'red')
    axes[2].axvline(upstream_t1, color='black', linestyle='--')
    axes[2].axvline(upstream_t2, color='black', linestyle='--')
    axes[2].axvline(downstream_t3, color='black', linestyle='--')
    axes[2].axvline(downstream_t4, color='black', linestyle='--')
    axes[2].set_ylabel('Velocity (km/s)', fontsize = 16)

    axes[3].plot(density.time, density.values, color='black')
    axes[3].axvline(date, c = 'red')
    axes[3].axvline(upstream_t1, color='black', linestyle='--')
    axes[3].axvline(upstream_t2, color='black', linestyle='--')
    axes[3].axvline(downstream_t3, color='black', linestyle='--')
    axes[3].axvline(downstream_t4, color='black', linestyle='--')
    axes[3].set_ylabel('Density (cm-3)', fontsize = 16)
    
    fig.suptitle(f'Plasma characteristics measured by: {spacecraft}', fontsize = 22)
    plt.tight_layout()
    
    #Create figure file name
    fig_filename = f'POSTER_PLOT_{spacecraft}_{date.year}_{date.month}_{date.day}.png'
    output_file = os.path.join('C:/Users/user/Desktop/', fig_filename)        #set directory to store file
    fig.savefig(output_file)
    plt.close()
    
    #Delete data arrays
    del B_field
    del B_field_comps
    del velocity
    del density
    
    
    return

In [9]:
#function to round a number to desised significant figures
def round_to_significant_figures(number, num_significant_figures):
    if number == 0:
        return 0
    return round(number, -int(math.floor(math.log10(abs(number)))) + (num_significant_figures - 1))

In [10]:
#Fucntion to get position of spacececraft in GSE coordinates.

def get_orbit(spacecraft, date):
    
    # Set time intervals
    start_time = date - dt.timedelta(minutes=1)
    
    end_time = date + dt.timedelta(minutes=1)
    
    # Read spacecraft;
    if spacecraft == 'THA':
        
        THA_orbit = spz.get_data(ssc_tree.Trajectories.themisa,
                         start_time, end_time, coordinate_system='gse')
        
        X = THA_orbit.filter_columns('X').values / 6378          #set in terms of Earth Radii in km
        Y = THA_orbit.filter_columns('Y').values / 6378
        Z = THA_orbit.filter_columns('Z').values / 6378

        #Round coordinates 
        position = [round_to_significant_figures(np.mean(X),3),round_to_significant_figures(np.mean(Y),3),round_to_significant_figures(np.mean(Z),3)]
             
        return position
    
    if spacecraft == 'THD':
        
        THD_orbit = spz.get_data(ssc_tree.Trajectories.themisd,
                         start_time, end_time, coordinate_system='gse')
        
        X = THD_orbit.filter_columns('X').values / 6378
        Y = THD_orbit.filter_columns('Y').values / 6378
        Z = THD_orbit.filter_columns('Z').values / 6378

        position = [round_to_significant_figures(np.mean(X),3),round_to_significant_figures(np.mean(Y),3),round_to_significant_figures(np.mean(Z),3)]
             
        return position
    
    if spacecraft == 'THE':
        
        THE_orbit = spz.get_data(ssc_tree.Trajectories.themise,
                         start_time, end_time, coordinate_system='gse')
        
        X = THE_orbit.filter_columns('X').values / 6378
        Y = THE_orbit.filter_columns('Y').values / 6378
        Z = THE_orbit.filter_columns('Z').values / 6378

        position = [round_to_significant_figures(np.mean(X),3),round_to_significant_figures(np.mean(Y),3),round_to_significant_figures(np.mean(Z),3)]
             
        return position
    
    else:
       
        print('Function Fail: spacecraft not recognised')
        
        return

Example Below

In [ ]:
#input desired date and spacecraft for test
date_entry = input('Input shock date in YYYY-MM-DD-HOUR-MINUTE-SECONDS format: ')
year, month, day, hour, minute, seconds = map(int, date_entry.split('-'))

shock_time = datetime(year, month, day, hour, minute, seconds)

spacecraft_str = input('Input spacecraft: ')

plot_data(spacecraft_str, shock_time)